In [62]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

% matplotlib inline

In [63]:
data = pd.read_csv("data/DOHMH_unique_inspections.csv")

In [65]:
data["INSPECTION DATE"] = pd.to_datetime(data["INSPECTION DATE"], infer_datetime_format=True)
data["GRADE DATE"] = pd.to_datetime(data["GRADE DATE"], infer_datetime_format=True)
data["RECORD DATE"] = pd.to_datetime(data["RECORD DATE"], infer_datetime_format=True)

In [66]:
# newestInspect = lambda x: x.loc[x["INSPECTION DATE"].argmax()]
data["NEWEST DATE"] = data.groupby("CAMIS")["INSPECTION DATE"].transform(max)

In [67]:
data = data[data["INSPECTION DATE"]==data["NEWEST DATE"]]

In [68]:
coffeeData = data[(data["CUISINE DESCRIPTION"]=="CafÃ©/Coffee/Tea")|
                  (data["CUISINE DESCRIPTION"]=="Donuts")].copy()

In [69]:
coffeeData["storetype"] = "other"
coffeeData.loc[coffeeData["DBA"].str.contains("dunkin.*donuts", case=False), "storetype"] = "dunkin"
coffeeData.loc[coffeeData["DBA"].str.contains("starbuck", case=False), "storetype"] = "starbucks"

In [70]:
coffeeData = coffeeData.reset_index(drop=True)
#coffeeData = coffeeData.drop("level_0", axis=1)


coffeeData = pd.concat([coffeeData, pd.get_dummies(coffeeData["storetype"])], axis=1)
#coffeeData.index.levels()

In [72]:
zipGroups = coffeeData.groupby("ZIPCODE")[["dunkin", "other", "starbucks"]].agg(np.sum)

In [73]:
# combine zip groups to match shape file: 11241 is within 11201, 
# and 11249/11211 are one shape in the shape file
zipGroups.loc[11201,:] = zipGroups.loc[11201,:] + zipGroups.loc[11241,:]
zipGroups.loc[11211,:] = zipGroups.loc[11211,:] + zipGroups.loc[11249,:]
zipGroups = zipGroups.drop([11241, 11249])

In [74]:
zipGroups["total"] = zipGroups.sum(axis=1)

In [75]:
for brand in ["dunkin", "starbucks", "other"]:
    zipGroups[brand + "_prop"] = zipGroups[brand]/zipGroups["total"]

In [76]:
zipGroups["total_prop"] = 1.0

In [77]:
zipGroups = zipGroups.reset_index().rename(index=str, columns={"ZIPCODE": "zip"})

In [79]:
import json
with open('data/nyc-zip-code.json') as json_file:
    zipshapes = json.load(json_file)

In [80]:
len(zipshapes["features"])

262

In [81]:
zipshapes = pd.DataFrame(map(lambda x: {"zip": x["properties"]["ZIP"],
                                    "area": x["properties"]["Shape_Area"]}, 
zipshapes["features"])).groupby("zip").agg(np.sum).reset_index()

In [82]:
zipshapes["zip"] = zipshapes["zip"].astype(int)

In [84]:
zipGroups = zipGroups.merge(zipshapes, on="zip", how="left")

In [85]:
zipGroups['area'] = zipGroups['area'] / 27878400.0

In [89]:
zipGroups.to_csv("data/coffee_per_zip.csv", float_format='%.5f')